In [5]:
import tensorflow as tf
import wandb
import dlomix
import os
import dlomix.losses
from dlomix.data import load_processed_dataset
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from dlomix.losses import masked_spectral_distance

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = '3'

In [3]:
MODEL_DIR = '/cmnfs/proj/bmpc_dlomix/models/baseline_models/noptm_baseline_full_bs1024_unmod_extended/'
RUN_NAME = '7ef3360f-2349-46c0-a905-01187d4899e2'
model = tf.keras.models.load_model(MODEL_DIR + RUN_NAME + '.keras',)
print(model.summary())

2024-07-24 07:33:32.563158: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7505 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:82:00.0, compute capability: 6.1


Model: "prosit_intensity_predictor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  464       
                                                                 
 sequential (Sequential)     (None, 30, 512)           1996800   
                                                                 
 sequential_1 (Sequential)   multiple                  4608      
                                                                 
 sequential_2 (Sequential)   (None, 29, 512)           1576806   
                                                                 
 encoder_att (AttentionLaye  multiple                  542       
 r)                                                              
                                                                 
 sequential_3 (Sequential)   multiple                  0         
                                        

In [7]:
ds = load_processed_dataset('/cmnfs/proj/bmpc_dlomix/datasets/processed/noptm_baseline_small_duplicated_ions')

In [8]:
ds.inference_only

False

In [9]:
ds.label_column

'intensities_raw'

In [21]:
type(ds.tensor_train_data)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [22]:
ds_test = load_processed_dataset('/cmnfs/proj/bmpc_dlomix/datasets/processed/noptm_baseline_small_bs1024_unmod_extended')

In [23]:
ds_test.label_column

In [2]:
from dlomix.data.dataset_config import DatasetConfig
from dlomix.data.dataset import PeptideDataset, load_from_disk
import importlib

In [10]:
path = '/cmnfs/proj/bmpc_dlomix/datasets/processed/noptm_baseline_small_bs1024_unmod_extended'
config = DatasetConfig.load_config_json(
    os.path.join(path, PeptideDataset.CONFIG_JSON_NAME)
)

In [11]:
config.inference_only

False

In [29]:
module = importlib.import_module("dlomix.data")
class_obj = getattr(module, config._additional_data.get("cls", "PeptideDataset"))

In [32]:
hf_dataset = load_from_disk(path)

In [40]:
hf_dataset

AttributeError: 'DatasetDict' object has no attribute 'label_column'

In [46]:
dataset = class_obj.from_dataset_config(config)

In [36]:
dataset.hf_dataset = hf_dataset

In [42]:
dataset.label_column